# Coverage Analysis (High Density Scenario)

_Script adapted from full-factorial.ipynb_

In [1]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import product, chain, combinations
from scipy import stats
from IPython.display import display, HTML
%matplotlib inline

def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

def get_file_name(name):
    return name.replace(':', '-')

## Config

In [2]:
inputFile = 'coverage.csv'
repetitionsCount = -1 # -1 = auto-detect
factors = ['R', 'T', 'm', 'D']

tIntervalAlpha = 0.9

plotSize = (10, 10)
plotStyle = 'seaborn-whitegrid'
saveFigures = False

# Filter scalars
scalarsFilter = ['Floorplan.userCount', 'Floorplan.coveredUsers:sum']
# Filter vectors
vectorsFilter = []
# Percentiles
percentiles = [0.25, 0.5, 0.75, 0.9, 0.95]

# Performance indexes
perfIndexes = [
    ('coveredUsersPercent', 'percentage of covered users')
]

# Transformations
transformations = [
]

intPercentiles = [int(i*100) for i in percentiles]
vecPerfIndexes = []
#for intPercentile in intPercentiles:
#    vecPerfIndexes.append(('broadcastTime' + str(intPercentile), 'Broadcast time needed to reach the ' + str(intPercentile) + 'th percentile of the coverage'))
for v in vecPerfIndexes:
    perfIndexes.append(v)
    #transformations.append((v[0], lambda x: math.log(x)))

## Load scalars

In [3]:
df = pd.read_csv('exported_data/' + inputFile, converters = {
    'attrvalue': parse_if_number,
    'binedges': parse_ndarray,
    'binvalues': parse_ndarray,
    'vectime': parse_ndarray,
    'vecvalue': parse_ndarray,
})

In [4]:
if repetitionsCount <= 0: # auto-detect
    repetitionsCount = int(df[df.attrname == 'repetition']['attrvalue'].max()) + 1
print('Repetitions:', repetitionsCount)

# Computed
factorsCount = len(factors)

if saveFigures:
    os.makedirs('figures', exist_ok=True)

Repetitions: 10


In [5]:
scalars = df[(df.type == 'scalar') | ((df.type == 'itervar') & (df.attrname != 'TO')) | ((df.type == 'param') & (df.attrname == 'Floorplan.userCount')) | ((df.type == 'runattr') & (df.attrname == 'repetition'))]
scalars = scalars.assign(qname = scalars.attrname.combine_first(scalars.module + '.' + scalars.name))
for index, row in scalars[scalars.type == 'itervar'].iterrows():
    val = scalars.loc[index, 'attrvalue']
    if isinstance(val, str)  and not all(c.isdigit() for c in val):
        scalars.loc[index, 'attrvalue'] = eval(val)
scalars.value = scalars.value.combine_first(scalars.attrvalue.astype('float64'))
scalars_wide = scalars.pivot_table(index=['run'], columns='qname', values='value')
scalars_wide.sort_values([*factors, 'repetition'], inplace=True)
count = 0
for index in scalars_wide.index:
    config = count // repetitionsCount
    scalars_wide.loc[index, 'config'] = config
    count += 1
scalars_wide = scalars_wide[['config', 'repetition', *factors, *scalarsFilter]]

configsCount = int(scalars_wide['config'].max()) + 1
totalSims = configsCount*repetitionsCount
display(HTML("<style>div.output_scroll { height: auto; max-height: 48em; }</style>"))
pd.set_option('display.max_rows', totalSims)
pd.set_option('display.max_columns', 100)

# coverage
scalars_wide['coveredUsersPercent'] = scalars_wide['Floorplan.coveredUsers:sum'] / (scalars_wide['Floorplan.userCount'] - 1)

## Load vectors

In [6]:
vectors = df[df.type == 'vector']
vectors = vectors.assign(qname = vectors.module + '.' + vectors.name)
for index in scalars_wide.index:
    r = index
    cfg = scalars_wide.loc[index, 'config']
    rep = scalars_wide.loc[index, 'repetition']
    vectors.loc[vectors.run == r, 'config'] = cfg
    vectors.loc[vectors.run == r, 'repetition'] = rep
vectors = vectors[vectors.qname.isin(vectorsFilter)]
vectors.sort_values(['config', 'repetition', 'qname'], inplace=True)
vectors = vectors[['config', 'repetition', 'qname', 'vectime', 'vecvalue']]

## Compute scalars from vectors

In [7]:
def get_percentile(percentile, vectime, vecvalue, totalvalue):
    tofind = percentile * totalvalue
    idx = 0
    csum = vecvalue.cumsum()
    for value in csum:
        if value >= tofind:
            return vectime[idx]
        idx += 1
    return math.inf

for index, row in vectors.iterrows():
    for vecPerf, percentile in zip(vecPerfIndexes, percentiles):
        vecPerfIndex = vecPerf[0]
        cfg = row['config']
        rep = row['repetition']
        if vecPerfIndex.startswith('broadcastTime'):
            total = scalars_wide[(scalars_wide['config'] == cfg) & (scalars_wide['repetition'] == rep)]['Floorplan.userCount'].values[0] - 1
        else:
            raise Exception('Need to specify total for ' + vecPerfIndex + '. (coding required)')
        value = get_percentile(percentile, row['vectime'], row['vecvalue'], total)
        scalars_wide.loc[(scalars_wide['config'] == cfg) & (scalars_wide['repetition'] == rep), vecPerfIndex] = value

## Apply transformations

In [8]:
for col, transform in transformations:
    scalars_wide[col] = scalars_wide[col].map(transform, 'ignore')

## Full factorial

In [9]:
for cfg in range(0, configsCount):
    for perfIndex, _ in perfIndexes:
        mean = scalars_wide[scalars_wide['config'] == cfg][perfIndex].mean()
        variance = scalars_wide[scalars_wide['config'] == cfg][perfIndex].var()
        _, positiveInterval = tuple(v*math.sqrt(variance/repetitionsCount) for v in stats.t.interval(tIntervalAlpha, repetitionsCount - 1))
        negerr = positiveInterval
        poserr = positiveInterval
        if perfIndex == 'coveredUsersPercent':
            poserr = min(1 - mean, positiveInterval)
        scalars_wide.loc[scalars_wide['config'] == cfg, perfIndex + 'Mean'] = mean
        scalars_wide.loc[scalars_wide['config'] == cfg, perfIndex + 'Variance'] = variance
        scalars_wide.loc[scalars_wide['config'] == cfg, perfIndex + 'Negerr'] = negerr
        scalars_wide.loc[scalars_wide['config'] == cfg, perfIndex + 'Poserr'] = poserr
scalars_wide = scalars_wide[scalars_wide['repetition'] == 0]

for perfIndex, _ in perfIndexes:
    del scalars_wide[perfIndex]
del scalars_wide['repetition']
del scalars_wide['Floorplan.userCount']
del scalars_wide['Floorplan.coveredUsers:sum']
del scalars_wide['config']

scalars_wide       

qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
RectangularCoverage-0-20210609-01:16:51-19735,1.0,1.0,1.0,1.0,0.000067,4.450376e-08,0.000122,0.000122
RectangularCoverage-300-20210609-01:23:29-20958,1.0,1.0,1.0,2.0,0.000067,4.450376e-08,0.000122,0.000122
RectangularCoverage-600-20210609-01:32:07-22182,1.0,1.0,1.0,3.0,0.000067,4.450376e-08,0.000122,0.000122
RectangularCoverage-900-20210609-01:39:57-23427,1.0,1.0,1.0,4.0,0.000133,1.780151e-07,0.000245,0.000245
RectangularCoverage-1200-20210609-01:47:45-24507,1.0,1.0,1.0,5.0,0.000000,0.000000e+00,0.000000,0.000000
RectangularCoverage-100-20210609-01:18:31-20197,1.0,1.0,2.0,1.0,0.000000,0.000000e+00,0.000000,0.000000
RectangularCoverage-400-20210609-01:25:50-21284,1.0,1.0,2.0,2.0,0.000000,0.000000e+00,0.000000,0.000000
RectangularCoverage-700-20210609-01:34:03-22564,1.0,1.0,2.0,3.0,0.000200,2.027394e-07,0.000261,0.000261
RectangularCoverage-1000-20210609-01:42:09-23731,1.0,1.0,2.0,4.0,0.000067,4.450376e-08,0.000122,0.000122


## Coverage results

Here we will print only the rows with a coverage mean > 0.99.

In [10]:
result = scalars_wide[scalars_wide['coveredUsersPercentMean'] >= 0.99]
sortedresult = result.sort_values(factors)
sortedresult

qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
RectangularCoverage-7460-20210609-04:19:40-14095,7.0,5.0,3.0,5.0,0.991394,2.654897e-05,0.002987,0.002987
RectangularCoverage-770-20210609-01:34:46-22714,8.0,1.0,2.0,3.0,0.992462,5.543191e-06,0.001365,0.001365
RectangularCoverage-1070-20210609-01:43:05-23869,8.0,1.0,2.0,4.0,0.991728,3.888640e-05,0.003615,0.003615
RectangularCoverage-1370-20210609-01:51:30-25043,8.0,1.0,2.0,5.0,0.994063,2.377984e-05,0.002827,0.002827
RectangularCoverage-870-20210609-01:37:27-23086,8.0,1.0,3.0,3.0,0.992061,6.670620e-06,0.001497,0.001497
RectangularCoverage-1170-20210609-01:45:59-24279,8.0,1.0,3.0,4.0,0.994997,7.244224e-06,0.001560,0.001560
RectangularCoverage-1470-20210609-01:54:15-25436,8.0,1.0,3.0,5.0,0.995063,7.733765e-06,0.001612,0.001612
RectangularCoverage-1970-20210609-02:06:55-27247,8.0,2.0,2.0,2.0,0.992061,1.557137e-05,0.002287,0.002287
RectangularCoverage-2270-20210609-02:14:20-28284,8.0,2.0,2.0,3.0,0.993662,1.654057e-05,0.002358,0.002358


In [11]:
for factor in factors:
    print('Minimize', factor)
    current = sortedresult[sortedresult[factor] == sortedresult[factor].min()]
    current = current.sort_values([factor, *[fac for fac in factors if fac != factor]])
    display(current)

Minimize R


qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
RectangularCoverage-7460-20210609-04:19:40-14095,7.0,5.0,3.0,5.0,0.991394,0.000027,0.002987,0.002987


Minimize T


qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
RectangularCoverage-770-20210609-01:34:46-22714,8.0,1.0,2.0,3.0,0.992462,0.000006,0.001365,0.001365
RectangularCoverage-1070-20210609-01:43:05-23869,8.0,1.0,2.0,4.0,0.991728,0.000039,0.003615,0.003615
RectangularCoverage-1370-20210609-01:51:30-25043,8.0,1.0,2.0,5.0,0.994063,0.000024,0.002827,0.002827
RectangularCoverage-870-20210609-01:37:27-23086,8.0,1.0,3.0,3.0,0.992061,0.000007,0.001497,0.001497
RectangularCoverage-1170-20210609-01:45:59-24279,8.0,1.0,3.0,4.0,0.994997,0.000007,0.001560,0.001560
RectangularCoverage-1470-20210609-01:54:15-25436,8.0,1.0,3.0,5.0,0.995063,0.000008,0.001612,0.001612
RectangularCoverage-680-20210609-01:32:45-22299,9.0,1.0,1.0,3.0,0.990594,0.000017,0.002373,0.002373
RectangularCoverage-980-20210609-01:41:10-23598,9.0,1.0,1.0,4.0,0.993529,0.000007,0.001526,0.001526
RectangularCoverage-1280-20210609-01:49:30-24757,9.0,1.0,1.0,5.0,0.996398,0.000002,0.000799,0.000799


Minimize m


qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
RectangularCoverage-680-20210609-01:32:45-22299,9.0,1.0,1.0,3.0,0.990594,0.000017,0.002373,0.002373
RectangularCoverage-980-20210609-01:41:10-23598,9.0,1.0,1.0,4.0,0.993529,0.000007,0.001526,0.001526
RectangularCoverage-1280-20210609-01:49:30-24757,9.0,1.0,1.0,5.0,0.996398,0.000002,0.000799,0.000799
RectangularCoverage-2780-20210609-02:28:08-30299,9.0,2.0,1.0,5.0,0.991928,0.000026,0.002976,0.002976
RectangularCoverage-690-20210609-01:33:06-22374,10.0,1.0,1.0,3.0,0.993996,0.000007,0.001525,0.001525
RectangularCoverage-990-20210609-01:41:33-23649,10.0,1.0,1.0,4.0,0.995597,0.000005,0.001279,0.001279
RectangularCoverage-1290-20210609-01:49:45-24790,10.0,1.0,1.0,5.0,0.997665,0.000002,0.000757,0.000757
RectangularCoverage-2190-20210609-02:13:00-28089,10.0,2.0,1.0,3.0,0.991127,0.000075,0.005026,0.005026
RectangularCoverage-2490-20210609-02:20:45-29200,10.0,2.0,1.0,4.0,0.993929,0.000014,0.002176,0.002176


Minimize D


qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
RectangularCoverage-4680-20210609-03:14:21-4586,9.0,4.0,2.0,1.0,0.992328,0.000006,0.001415,0.001415
RectangularCoverage-4780-20210609-03:17:06-4968,9.0,4.0,3.0,1.0,0.991661,0.000008,0.001633,0.001633
RectangularCoverage-6180-20210609-03:49:23-9720,9.0,5.0,2.0,1.0,0.990060,0.000007,0.001541,0.001541
RectangularCoverage-6280-20210609-03:52:39-10168,9.0,5.0,3.0,1.0,0.990460,0.000012,0.002047,0.002047
RectangularCoverage-1690-20210609-02:00:21-26295,10.0,2.0,2.0,1.0,0.990193,0.000013,0.002087,0.002087
RectangularCoverage-1790-20210609-02:02:48-26646,10.0,2.0,3.0,1.0,0.991861,0.000006,0.001467,0.001467
RectangularCoverage-3190-20210609-02:38:09-31848,10.0,3.0,2.0,1.0,0.991795,0.000010,0.001842,0.001842
RectangularCoverage-3290-20210609-02:41:22-32303,10.0,3.0,3.0,1.0,0.992795,0.000006,0.001456,0.001456
RectangularCoverage-4790-20210609-03:17:20-5003,10.0,4.0,3.0,1.0,0.993662,0.000010,0.001788,0.001788


## Observations

As we can see, we need at least R = 7m to get a mean coverage of 99%.

With R = 7m, the lowest config is: R=7m, T=1s, m=2, D=5s. D can be lowered to 3s if we increment T to 4s and m to 3.

With R = 8m we get a lot more possible configurations, but m needs to be at least 2 and D at least 3s. If we use R = 9m, we can have m = 1 and T = 1s but D must be at least 3s.

A "good" and "balanced" config, that does not use the value `1` for any of the parameters, is the one displayed below. From now on, for the high density scenario, we will consider the following as minimum parameters required to get the 99% coverage.

R = 8m

T = 2s

m = 2

D = 2s

In [12]:
display(sortedresult[(sortedresult['R'] == 8) & (sortedresult['T'] == 2) & (sortedresult['m'] == 2) & (sortedresult['D'] == 2)])

qname,R,T,m,D,coveredUsersPercentMean,coveredUsersPercentVariance,coveredUsersPercentNegerr,coveredUsersPercentPoserr
run,,,,,,,,
RectangularCoverage-1970-20210609-02:06:55-27247,8.0,2.0,2.0,2.0,0.992061,0.000016,0.002287,0.002287
